In [14]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdb


# import bokeh
# from bokeh.io import output_notebook, show
# from bokeh.resources import INLINE
# import bokeh.plotting as bp
# from bokeh.plotting import figure
#output_notebook(resources=INLINE)



mpl.style.use('seaborn')
mpl.rcParams['figure.figsize'] = [20,15]
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['xtick.labelsize'] =15
mpl.rcParams['ytick.labelsize'] = 15
mpl.rcParams['legend.fontsize'] = 15
pd.options.mode.chained_assignment = None


PLT_WIDTH_INCH = 20
PLT_HEIGHT_INCH = 15
plt.rcParams['figure.figsize'] = [PLT_WIDTH_INCH, PLT_HEIGHT_INCH]

%load_ext autoreload
%matplotlib inline



#task specific modules
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# %load spotify_client/spotify_client.py
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


"""
we will get all the features from spotify, by song name and artist name.
this function will create a json file with all the data
"""
def get_song_features_and_info_json(song_name, artist_name):

    # getting credentials to request special API
    credenials = SpotifyClientCredentials(client_id="5659eaf41b194134866170761e2fb293",client_secret="17e2a9fa37c2466baa2ba05752896cdc")

    # getting the client
    spotify_client = spotipy.Spotify(client_credentials_manager=credenials)

    # getting the song id in order to retrieve features, limiting to 1 since it sorts the results by popularity
    song_results = spotify_client.search(q='artist:' + artist_name + ' track:' + song_name, type='track',limit=1)
    #print(song_results)
    tracks = song_results['tracks']['items']
    if len(tracks)==0:
        print("***no results for the requested track:{}***".format(song_name+":"+artist_name))
        return
    result_dict = tracks[0]

    # creating dict for track
    song_dict = creating_song_dictionary(artist_name, result_dict, spotify_client)

    return song_dict
def creating_song_dictionary(artist_name, result_dict, spotify_client):
    song_dict = {}
    song_dict['artist'] = artist_name
    song_id = [result_dict['uri']]
    song_dict['uri'] = song_id
    song_dict['year'] = result_dict['album']['release_date'][:4]
    song_dict['artist_id'] = result_dict['artists'][0]['id']
    song_dict['track_id'] = result_dict['id']
    song_dict['name'] = spotify_client.track(song_dict['track_id'])['name']
    song_dict['popularity'] = result_dict['popularity']
    song_dict['preview_link'] = result_dict['preview_url']
    features = spotify_client.audio_features(song_id)
    song_dict['main_features'] = features[0]

    # todo - need this? a big analysis, maybe will be helpful
    # analysis = spotify_client._get(features[0]['analysis_url'])
    #
    # song_dict['analysis'] = analysis

    return song_dict

In [8]:

get_song_features_and_info_json("Dont Let The Sun Catch You Crying","Gerry and The Pacemakers")

{'artist': 'Gerry and The Pacemakers',
 'artist_id': '3UmBeGyNwr4iDWi1vTxWi8',
 'main_features': {'acousticness': 0.406,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ZMMtH875IR2TfkyC4PolD',
  'danceability': 0.477,
  'duration_ms': 157093,
  'energy': 0.352,
  'id': '0ZMMtH875IR2TfkyC4PolD',
  'instrumentalness': 0,
  'key': 1,
  'liveness': 0.122,
  'loudness': -14.165,
  'mode': 1,
  'speechiness': 0.03,
  'tempo': 106.773,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/0ZMMtH875IR2TfkyC4PolD',
  'type': 'audio_features',
  'uri': 'spotify:track:0ZMMtH875IR2TfkyC4PolD',
  'valence': 0.478},
 'name': "Don't Let the Sun Catch You Crying - Mono; Main",
 'popularity': 39,
 'preview_link': 'https://p.scdn.co/mp3-preview/45cc5668aa48b33280bf44354ea4742d2d048f44?cid=5659eaf41b194134866170761e2fb293',
 'track_id': '0ZMMtH875IR2TfkyC4PolD',
 'uri': ['spotify:track:0ZMMtH875IR2TfkyC4PolD'],
 'year': '2008'}

In [11]:
df=pd.read_pickle('billboard.dat')

In [20]:
df['lyrics'][0]

"Mona Lisa, Mona Lisa, men have named you\nYou're so like the lady with the mystic smile\nIs it only 'cause you're lonely they have blamed you?\nFor that Mona Lisa strangeness in your smile?\n\nDo you smile to tempt a lover, Mona Lisa?\nOr is this your way to hide a broken heart?\nMany dreams have been brought to your doorstep\nThey just lie there and they die there\n\nAre you warm, are you real, Mona Lisa?\nOr just a cold and lonely lovely work of art?\n\nDo you smile to tempt a lover, Mona Lisa?\nOr is this your way to hide a broken heart?\nMany dreams have been brought to your doorstep\nThey just lie there and they die there\n\nAre you warm, are you real, Mona Lisa?\nOr just a cold and lonely lovely work of art?\n\nMona Lisa, Mona Lisa"